In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import os

# Define the datasets
BASE_DIR = os.getcwd()  # or specify your base directory
datasets = [
    {'name': 'A_Flood_Dataset', 'path': os.path.join(BASE_DIR, 'datasets', 'A_Flood_Dataset.csv')},
    {'name': 'B1_Monthly_Rainfall', 'path': os.path.join(BASE_DIR, 'datasets', 'B1_Monthly_Rainfall.csv')},
    {'name': 'B2_Monthly_Rainfall', 'path': os.path.join(BASE_DIR, 'datasets', 'B2_Monthly_Rainfall.csv')}
]

# Create widgets
dataset_dropdown = widgets.Dropdown(
    options=[(ds['name'], ds['path']) for ds in datasets],
    description='Dataset:',
    disabled=False,
    style={'description_width': 'initial'}
)

preview_output = widgets.Output()
confirm_button = widgets.Button(description='Confirm Selection', disabled=False)
confirm_output = widgets.Output()

# Define a mapping of datasets to columns to drop
columns_to_drop = {
    'A_Flood_Dataset': ['Sl', 'Station_Names', 'Station_Number', 'X_COR', 'Y_COR', 'Period', 'Flood?'],  # Replace with actual column names
    'B1_Monthly_Rainfall': ['unwanted_col1', 'unwanted_col2'],  # Replace with actual column names
    'B2_Monthly_Rainfall': ['irrelevant_col1', 'irrelevant_col2']  # Replace with actual column names
}

# Define functions for interactivity
def on_dataset_change(change):
    global selected_dataset
    selected_dataset = change['new']
    with preview_output:
        preview_output.clear_output()
        try:
            df = pd.read_csv(selected_dataset)
            print(df.head())
        except Exception as e:
            print(f"Error loading dataset: {e}")

def on_confirm_click(b):
    with confirm_output:
        confirm_output.clear_output()
        try:
            df = pd.read_csv(selected_dataset)
            print(f"✅ Dataset confirmed: {selected_dataset}")
            print("Preview of selected dataset:")
            
            # Get the selected dataset name from the dropdown
            selected_dataset_name = dataset_dropdown.value  # This gets the path, we need the name
            global dataset_name
            dataset_name = next(ds['name'] for ds in datasets if ds['path'] == selected_dataset)
            
            # Drop specified columns based on the selected dataset
            if dataset_name in columns_to_drop:
                df.drop(columns=columns_to_drop[dataset_name], inplace=True, errors='ignore')
                print(f"Dropped columns: {columns_to_drop[dataset_name]}")
            print(dataset_name)
            display(df.head())
            
            # Store the selected dataframe in a global variable for the next cell
            global selected_dataframe
            selected_dataframe = df
            
        except Exception as e:
            print(f"Error confirming dataset: {e}")

# Set up event handlers
dataset_dropdown.observe(on_dataset_change, names='value')
confirm_button.on_click(on_confirm_click)

# Initial load
selected_dataset = datasets[0]['path']
on_dataset_change({'new': selected_dataset})

# Display the widgets
display(widgets.VBox([
    dataset_dropdown,
    widgets.HTML('<h3 style="margin-top:20px">Dataset Preview:</h3>'),
    preview_output,
    confirm_button,
    widgets.HTML('<hr style="margin:20px 0">'),
    confirm_output
]))


In [3]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

# Function to create input fields for a selected row
def create_input_form(row):
    # Clear previous output
    input_output.clear_output()
    
    # Create input fields for each column in the row
    input_fields = []
    for col in row.index:
        # Use a fixed width for the description/label
        description_layout = Layout(width='200px')  # Adjust this value as needed
        
        if isinstance(row[col], (int, float)):  # Numeric types
            input_fields.append(widgets.FloatText(
                value=row[col],
                description=col,
                disabled=False,
                layout=Layout(width='auto'),
                style={'description_width': '100px'}  # This makes the label use the full width
            ))
        else:  # Non-numeric types (e.g., strings)
            input_fields.append(widgets.Text(
                value=str(row[col]),
                description=col,
                disabled=False,
                layout=Layout(width='auto'),
                style={'description_width': '100px'}  # This makes the label use the full width
            ))
    
    # Split into 2 columns with proper spacing
    column1 = widgets.VBox(input_fields[::2], layout=Layout(width='50%', padding='0 10px 0 0'))
    column2 = widgets.VBox(input_fields[1::2], layout=Layout(width='50%', padding='0 0 0 10px'))
    
    # Create the predict button with some top margin
    predict_button = widgets.Button(
        description="Predict Flood Status",
        layout=Layout(margin='20px 0 0 0'),
        button_style='warning'
    )
    
    # Function to handle button click - LEAVE THIS FOR YOUR IMPLEMENTATION
    def on_predict(b):
        with input_output:
            input_output.clear_output()

            # Extract the first row from selected_dataframe
            row = selected_dataframe.iloc[0]
            
            # Convert row to a dictionary with lowercase keys and float conversion where possible
            converted_input_data = {}
            for key, value in row.items():
                try:
                    converted_input_data[key.lower()] = float(value)
                except (ValueError, TypeError):
                    converted_input_data[key.lower()] = value

            # Collect user inputs from input_fields
            input_data = {col: input_fields[i].value for i, col in enumerate(row.index)}

            # Convert user inputs to float where possible
            input_data_converted = {}
            for key, value in input_data.items():
                try:
                    input_data_converted[key.lower()] = float(value)
                except (ValueError, TypeError):
                    input_data_converted[key.lower()] = value

            # Define expected columns (replace with your actual trained model's feature columns)
            # For demonstration, assuming the columns are same as in the first row but in lowercase
            expected_columns = [col.lower() for col in row.index]

            # Create input DataFrame with columns matching the model's expected features
            input_df = pd.DataFrame([input_data_converted], columns=expected_columns)

            # Determine model filename based on dataset
            if dataset_name == 'A_Flood_Dataset':
                model_name = 'ML_verA_trained.pkl'
            elif dataset_name == 'B1_Monthly_Rainfall':
                model_name = 'ML_verB1_trained.pkl'
            elif dataset_name == 'B2_Monthly_Rainfall':
                model_name = 'ML_verA_trained.pkl'
            else:
                model_name = None  # Handle unexpected dataset

            if model_name:
                import os
                import joblib

                model_path = os.path.join(BASE_DIR, 'src', 'ML', 'trained_model', model_name)
                loaded_model = joblib.load(model_path)

                # Make prediction
                prediction = loaded_model.predict(input_df)

                # Example logic for flood prediction
                numeric_values = [val for val in input_data_converted.values() if isinstance(val, (int, float))]
                total_value = sum(numeric_values)
                flood_prediction = "Likely" if total_value > 50 else "Unlikely"
                confidence = "High" if abs(total_value - 50) > 25 else "Medium"

                # Display the prediction result
                display(widgets.HTML(
                    f"""<div style='padding: 20px; background-color: #f8f9fa; border-radius: 5px; border-left: 5px solid {'#dc3545' if flood_prediction == 'Likely' else '#28a745'};'>
                        <h3 style='margin-top: 0;'>Flood Prediction Result</h3>
                        <p><strong>Prediction:</strong> <span style='color: {'#dc3545' if flood_prediction == 'Likely' else '#28a745'}'>Flood {flood_prediction}</span></p>
                        <p><strong>Confidence:</strong> {confidence}</p>
                        <p><strong>Details:</strong></p>
                        <div style='display: flex; flex-wrap: wrap;'>""" +
                        "".join([f"<div style='flex: 1 0 200px; margin-bottom: 5px;'><strong>{col}:</strong> {val}</div>" 
                                for col, val in input_data.items()]) +
                        "</div></div>"
                    )
                )
            else:
                # Handle case where model filename is not set
                display(widgets.HTML("<p>Error: Unknown dataset or model not found.</p>"))
    
    predict_button.on_click(on_predict)
    
    # Display everything with proper spacing
    with input_output:
        display(widgets.VBox([
            widgets.HBox([column1, column2], layout=Layout(width='100%')),
            predict_button
        ]))

# Create an output area for the input form
input_output = widgets.Output()

# Create a dropdown to select a row from the selected_dataframe
row_selector = widgets.Dropdown(
    options=[(f"Row {i}", i) for i in range(len(selected_dataframe))],
    description='Select Row:',
    disabled=False,
    layout=Layout(width='300px')  # Make the dropdown wider if needed
)

# Function to update the input form when a row is selected
def on_row_change(change):
    selected_row_index = change['new']
    selected_row = selected_dataframe.iloc[selected_row_index]
    create_input_form(selected_row)

# Set up event handler for row selection
row_selector.observe(on_row_change, names='value')

# Display the row selector and the input output area
display(widgets.VBox([
    widgets.HTML("<h3>Flood Prediction System</h3>"),
    row_selector,
    input_output
]))

# Load the first row by default
create_input_form(selected_dataframe.iloc[0])
